In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pickle

In [2]:
#Load the data
data = pd.read_csv('Churn_Modelling.csv')
print('Shape of the data is: ', data.shape)
data.head()

Shape of the data is:  (10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Preprocess the data
# Drop irrelevant columns

data = data.drop(columns=['RowNumber', 'CustomerId', 'Surname'])
print('Shape of the data after dropping columns:', data.shape)
data.head()

Shape of the data after dropping columns: (10000, 11)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# Encode categorical data

data['Gender'].value_counts()

Gender
Male      5457
Female    4543
Name: count, dtype: int64

In [5]:
label_encoder = LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])
data['Gender'].value_counts()

Gender
1    5457
0    4543
Name: count, dtype: int64

In [6]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [7]:
data['Geography'].value_counts()

# We won't use label encoding for this dataset because Geography has  multiple values and each value will have 1,2,3 like this. The model can think 3 has higher value than 1 and 2 and vice versa.

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [8]:
# One-hot encoding for Geography column

from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder()

one_hot_encoded_data = one_hot_encoder.fit_transform(data[['Geography']])

In [9]:
one_hot_encoded_data.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [10]:
one_hot_encoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [11]:
geo_one_hot_encoded = pd.DataFrame(data = one_hot_encoded_data.toarray(), columns=one_hot_encoder.get_feature_names_out(['Geography']))
geo_one_hot_encoded.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [12]:
# Combine one hot encoded columns with original data

data = pd.concat([data.drop(columns=['Geography']), geo_one_hot_encoded], axis=1)

data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [13]:
# Save the encoders and scalers

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder, file)

In [14]:
# Divide the data into independent and dependent variables

data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [15]:
X = data.drop(['Exited'], axis=1)
y = data['Exited']

# Splitting the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print('Xtrain shape:', X_train.shape)
print('Xtest shape:', X_test.shape)
print('ytrain shape:', y_train.shape)
print('ytest shape:', y_test.shape)

Xtrain shape: (8000, 12)
Xtest shape: (2000, 12)
ytrain shape: (8000,)
ytest shape: (2000,)


In [16]:
X_train

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
7389,667,0,34,5,0.00,2,1,0,163830.64,0.0,0.0,1.0
9275,427,1,42,1,75681.52,1,1,1,57098.00,0.0,1.0,0.0
2995,535,0,29,2,112367.34,1,1,0,185630.76,1.0,0.0,0.0
5316,654,1,40,5,105683.63,1,1,0,173617.09,0.0,0.0,1.0
356,850,0,57,8,126776.30,2,1,1,132298.49,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9225,594,0,32,4,120074.97,2,1,1,162961.79,0.0,1.0,0.0
4859,794,0,22,4,114440.24,1,1,1,107753.07,0.0,0.0,1.0
3264,738,1,35,5,161274.05,2,1,0,181429.87,1.0,0.0,0.0
9845,590,0,38,9,0.00,2,1,1,148750.16,0.0,0.0,1.0


In [17]:
## Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [18]:
X_train

array([[ 0.16958176, -1.09168714, -0.46460796, ..., -1.01460667,
        -0.5698444 ,  1.74309049],
       [-2.30455945,  0.91601335,  0.30102557, ..., -1.01460667,
         1.75486502, -0.57369368],
       [-1.19119591, -1.09168714, -0.94312892, ...,  0.98560362,
        -0.5698444 , -0.57369368],
       ...,
       [ 0.9015152 ,  0.91601335, -0.36890377, ...,  0.98560362,
        -0.5698444 , -0.57369368],
       [-0.62420521, -1.09168714, -0.08179119, ..., -1.01460667,
        -0.5698444 ,  1.74309049],
       [-0.28401079, -1.09168714,  0.87525072, ..., -1.01460667,
         1.75486502, -0.57369368]])

In [19]:
# Save scaler
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [20]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


## ANN Implementation

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [22]:
X_train.shape[1] # 12 features in input layer

12

In [23]:
# Build ANN model

model = Sequential(
    [
        Dense(64, activation="relu", input_shape=(X_train.shape[1],)),  # HL1 Connected with input layet
        Dense(32, activation="relu"),  # HL2 Connected with HL1
        Dense(1, activation="sigmoid") # Output layer
    ]
)

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
# When we want to update any hyperparameter witb respect to optimizers or loss, create seperate objects and pass the value. Otherwise directly use 'adam' or 'binary_crossentropy' as a string.
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [26]:
# Compile the model

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
# Setup the Tensorboard

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [28]:
# Setup Early Stopping

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [29]:
# Train the model

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100,
                    callbacks=[tensorflow_callback, early_stopping_callback])

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3993 - accuracy: 0.8360 - val_loss: 0.3655 - val_accuracy: 0.8405
Epoch 2/100
250/250 [==============================] - 0s 668us/step - loss: 0.3566 - accuracy: 0.8556 - val_loss: 0.3447 - val_accuracy: 0.8635
Epoch 3/100
250/250 [==============================] - 0s 665us/step - loss: 0.3504 - accuracy: 0.8558 - val_loss: 0.3483 - val_accuracy: 0.8540
Epoch 4/100
250/250 [==============================] - 0s 708us/step - loss: 0.3425 - accuracy: 0.8599 - val_loss: 0.3502 - val_accuracy: 0.8580
Epoch 5/100
250/250 [==============================] - 0s 729us/step - loss: 0.3430 - accuracy: 0.8606 - val_loss: 0.3566 - val_accuracy: 0.8470
Epoch 6/100
250/250 [==============================] - 0s 636us/step - loss: 0.3393 - accuracy: 0.8597 - val_loss: 0.3451 - val_accuracy: 0.8545
Epoch 7/100
250/250 [==============================] - 0s 670us/step - loss: 0.3373 - accuracy: 0.8599 - val_loss: 0.3549 - val_accu

In [30]:
model.save('model.h5')

/Users/nkejriwal/Documents/Learning/DL_ANN/learn_env/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [31]:
# Load Tensorboard Extension
%load_ext tensorboard



In [32]:
%tensorboard --logdir logs/fit

ERROR: Failed to launch TensorBoard (exited with 126).
Contents of stderr:
/Users/nkejriwal/Documents/Learning/DL_ANN/learn_env/bin/tensorboard: line 2: /Users/nkejriwal/Documents/Personal Docs/ML_NLP_Course/DL_ANN/learn_env/bin/python3.11: No such file or directory
/Users/nkejriwal/Documents/Learning/DL_ANN/learn_env/bin/tensorboard: line 2: exec: /Users/nkejriwal/Documents/Personal Docs/ML_NLP_Course/DL_ANN/learn_env/bin/python3.11: cannot execute: No such file or directory